In [1]:
# Initialize seed: The created seed can be saved and loaded in again to continue working on the generated profile. This way the 
# profile does not change each time the simulation is performed again. 

#from emobpy.tools import set_seed
#set_seed(dir=r'C:\Users\janva\Emobpy\example\config_files/')

#Import packages 
from emobpy import Mobility
from emobpy.constants import RULE; RULE
from emobpy import DataBase
from emobpy import Consumption, HeatInsulation, BEVspecs
from emobpy import Availability
from emobpy import Charging
from emobpy import Export
import os

In [2]:
# Set simulation parameters for the loop 

activity = 'inactive'                                                             #Activity types = working/inactive
EVs = 2                                                                           #Number of profiles that need to be generated in the loop
EV_type = 'RZ'                                                                    #Renault Zoe (RZ), Tesla model 3 (T3), Tesla model S (TS), Hyundai Kona (HK)

folder = 1                                                                        #First folder that is generated. Usefull when you already have 10 profiles generated 
                                                                                  #but you want to add more, change the folder to 11 to start from there

# The data of each EV that is generated will be stored in a seperate folder (RZ, T3, T3 or HK in this case). In this folder
# different sub folders will be created with the activity type (working/inactive) followed by the number of the iteration. 

In [4]:
while (folder<=EVs):
    path = r'C:\Users\janva\Emobpy\example\db\test'                               #Set path for output files 
    file_path = os.path.join(path,EV_type,activity+str(folder))
    inactive_working = os.path.join("DepartureDestinationTrip_"+activity+".csv")
    
    m = Mobility(config_folder=r'C:\Users\janva\Emobpy\example\config_files/')    #Emobpy mobility profile based on these probability distributions for trip duration, distance, destination and departure time

    m.set_params(
                 name_prefix="BEV1",                  
                 total_hours=7*24,                                                #One week for this example
                 time_step_in_hrs=0.25,                                           #15 minute time steps
                 category="user_defined",
                 reference_date="01/01/2016"                                      #Weather data based on this date 
                 )

    m.set_stats(
                stat_ntrip_path="TripsPerDay.csv",                                #Number of trips per day
                stat_dest_path=inactive_working,                                  #Dependent on working or inactive owner 
                stat_km_duration_path="DistanceDurationTrip.csv",                 #Trip distance and duration probability distribution 
                )

    m.set_rules(rule_key="user_defined")                                          # see /config_files/rules.yml, it contains a dictionary
                                                                                  # whose key must be the same as rule_key. 
    m.run()


    m.save_profile(folder=file_path)                                              # Emobpy mobility profile saved to database

    #Mobility dataframes
    mobility_TS = m.timeseries
    mobility_profile = m.profile

    #Database 
    DB = DataBase(file_path)
    DB.loadfiles()
    DB.db.keys()


    ## Driving consumption time series
    DB.update()                                                                   # This load new files hosted in database folder as result of new generated files
    mname = m.name
    HI = HeatInsulation(True)                                                     # Creating the heat insulation by copying the default configuration
    BEVS = BEVspecs()                                                             # Database that contains BEV models

    # Select the electric vehicle chosen in the simulation parameters from the first cell. 
    # Many other EVs are preloaded into Emobpy. 
    
    R_Zoe = BEVS.model(('Renault','Zoe Q90',2019))
    #Tesla_3 = BEVS.model(('Tesla','Model 3 Standard Range RWD',2019))
    #Tesla_S = BEVS.model(('Tesla','Model S 60 RWD',2015))
    #H_Kona = BEVS.model(('Hyundai','KONA Electric 64 kWh',2019))


    c = Consumption(mname, R_Zoe)                                                 # Do not forget to change the name from R_Zoe to any other chosen EV
    c.load_setting_mobility(DB)

    #from emobpy import Weather                                                   # Import other weather files form different countries if desired 
    #WD = Weather()
    #WD.download_weather_data()

    c.run(
      heat_insulation=HI,
      weather_country='NL',                # Select weather file 
      weather_year=2016,                   # Year
      passenger_mass=75,                   # kg
      passenger_sensible_heat=70,          # W
      passenger_nr=1.5,                    # Passengers per vehicle including driver
      air_cabin_heat_transfer_coef=20,     # W/(m2K). Interior walls
      air_flow = 0.02,                     # m3/s. Ventilation
      driving_cycle_type='WLTC',           # Two options "WLTC" or "EPA"
      road_type=0,                         # For rolling resistance, Zero represents a new road.
      road_slope=0
      )

    c.save_profile(file_path)

    #Create dataframes 
    consumption_TS = c.timeseries                                                 # Consumption in kWh/timestep -> timestep 15 min in this example
    consumption_profile = c.profile.head()


    ## Grid availability time series 
    DB.update()                                                                   # This load new generated files that are hosted in database folder
    cname = c.name                                                                # Getting the id of the first consumption profile


    station_distribution = {                                                      # Dictionary with charging stations type probability distribution per the purpose of the trip (location or destination)
                            'prob_charging_point': {
                                'errands': {'public': 0.8, 'none': 0.2},
                                'escort': {'public': 0.8, 'none': 0.2},
                                'leisure': {'public': 0.8, 'none': 0.2},
                                'shopping': {'public': 0.8, 'none': 0.2},
                                'home': {'public': 0.0, 'none': 0.2, 'home': 0.8},
                                'workplace': {'public': 0.0, 'workplace': 0.0, 'none': 1.0},   # If the vehicle is at the workplace, it will always find a charging station available (assumption)
                                'driving': {'none': 0.99, 'fast75': 0.005, 'fast150': 0.005}}, # with the low probability given to fast charging is to ensure fast charging only for very long trips (assumption)
                            'capacity_charging_point': {                                       # Nominal power rating of charging station in kW
                                                        'public': 22,
                                                        'home': 3.7,
                                                        'workplace': 11,
                                                        'none': 0,  # dummy station
                                                        'fast75': 150,
                                                        'fast150': 150}
                            }

    ga = Availability(cname, DB)
    ga.set_scenario(station_distribution)


    ga.run()
    ga.save_profile(file_path)

    #Create dataframes 
    availability_TS = ga.timeseries
    availability_profile = ga.profile


    ## Grid electricity demand 
    DB.update()

    aname = ga.name                                                               # Getting the id of the availability profile

    strategies = [
                  "immediate",                                                    # When battery has SOC < 100% then it charges immediatelly at a maximun power rating of the current charging station
                  "balanced",                                                     # When battery has SOC < 100% then it charges immediatelly but at lower rating power to ensure 100% SOC at the end (before moving to another place).
                  "from_0_to_24_at_home",                                         # Customized: starting time of charging (this case 0 hrs), final time of charging (this case 24 hrs), at could be one 'location' (this case 'home') or 'any'.
                  "from_23_to_8_at_any"                                           # Night charging strategy 
                  ]

    for option in strategies:
        ged = Charging(aname)
        ged.load_scenario(DB)
        ged.set_sub_scenario(option)
        ged.run()
        print(f'Creation Successful:{ged.success}')                               # if False, modify the strategy to a less constrained.
        ged.save_profile(file_path)
    
    charging_TS = ged.timeseries
    charging_profile = ged.profile

    #All data in one dataframe 
    file_path_alldata = os.path.join(path,EV_type,activity+str(folder),'All_data.csv')
    All_data = charging_TS.join(availability_TS[["soc"]])
    All_data = All_data.join(consumption_TS[["average power in W"]])
    All_data.to_csv(file_path_alldata)

    print (folder)                                                                #Print itteration of loop 
    DB.update()
    
    #Export the charging profile of the EV and the input data of the EV 
    Exp = Export()
    Exp.loaddata(DB)
    Exp.to_csv()
    Exp.save_files(repository=file_path)
    folder = folder+1

New profile running: BEV1_W2_4bb05
Progress: 0% [0 / 7] days

C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\mobility.py:644: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  npdf2.loc[:, "days"] = npdf2["days"].apply(self.uniquedays.index)
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\mobility.py:645: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  npdf2.loc[:, "purpose"] = npdf2["purpose"].apply(self.states.index)


Progress: 57% [4 / 7] days

C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\mobility.py:1187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.logdf = self.logdf.append(
C:\Users\janva\anaco

Progress: 100% [7 / 7] days
Profile done: BEV1_W2_4bb05
Elapsed time (min): 0.2
 


...

C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\consumption.py:1959: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  rp.loc[:, repeats] = rp[repeats].fillna(method="ffill")
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\consumption.py:1960: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  rp.loc[:, fixed] = rp[fixed].fillna(0)
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\consumption.py:1971: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting

 
See Log files
C:\Users\janva\Emobpy\notebook\log\emobpy.log
soc_init:0.5 --> soc_end:1.0
Profile done: BEV1_W2_99e3e_Renault_Zoe_Q90_2019_b75e3_avai_6ab82
 
See Log files
C:\Users\janva\Emobpy\notebook\log\emobpy.log


C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\availability.py:458: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.rp.loc[:, self.repeats] = self.rp[self.repeats].fillna(method="ffill")
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\availability.py:459: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.rp.loc[:, self.fixed] = self.rp[self.fixed].fillna(0)
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\availability.py:464: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the 

Balance: -0.01. Consumption: 39.01. charge_battery: 61.81. sto_init: 22.8. sto_end: 45.61
Profile done: BEV1_W2_99e3e_Renault_Zoe_Q90_2019_b75e3_avai_6ab82_immediate_e963c
Creation Successful:True
Balance: -0.01. Consumption: 39.01. charge_battery: 61.81. sto_init: 22.8. sto_end: 45.61
Profile done: BEV1_W2_99e3e_Renault_Zoe_Q90_2019_b75e3_avai_6ab82_balanced_a8bc9
Creation Successful:True
Balance: -0.01. Consumption: 39.01. charge_battery: 61.81. sto_init: 22.8. sto_end: 45.61
Profile done: BEV1_W2_99e3e_Renault_Zoe_Q90_2019_b75e3_avai_6ab82_from_0_to_24_at_home_b3991
Creation Successful:True
Balance: -0.01. Consumption: 39.01. charge_battery: 61.81. sto_init: 22.8. sto_end: 45.61
Profile done: BEV1_W2_99e3e_Renault_Zoe_Q90_2019_b75e3_avai_6ab82_from_23_to_8_at_any_a1810
Creation Successful:True
2
Summary file: C:\Users\janva\Emobpy\example\db\test\RZ\inactive2\bev_data_input.csv
Time series file: C:\Users\janva\Emobpy\example\db\test\RZ\inactive2\bev_time_series.csv


C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\export.py:76: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "hh"] = df["hh"].astype(int)
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\export.py:76: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "hh"] = df["hh"].astype(int)
C:\Users\janva\anaconda3\envs\emob\lib\site-packages\emobpy\export.py:148: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behav